In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
!pip install requests

In [8]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import requests
import xml.etree.ElementTree as ET

In [7]:
import xml.dom.minidom 

In [6]:
import sqlite3

In [5]:
from bs4 import BeautifulSoup

In [4]:
def fetch_rss_feed(feed_url):
    response = requests.get(feed_url)
    if response.status_code == 200:
        return response.content
    else:
        print(f"Failed to fetch RSS feed.")
        return None

In [33]:
def parse_and_insert(data, feed_type): 
    soup = BeautifulSoup(data, 'xml')

    feed_datas = []

    for item in soup.find_all('item'):
        #print(item.find('description').text)
        title = item.find('title').text.strip() if item.find('title') is not None else ''

        description_element = item.find('description')
        description = description_element.text if description_element is not None else ''

        link = item.find('link').text.strip() if item.find('link') is not None else ''
        pubDate = item.find('pubDate').text.strip() if item.find('pubDate') is not None else ''
        imgLink = item.find('enclosure')['url']
    
#         print(f"Title: {title}")
#         print(f"Description: {description}")
#         print(f"Link: {link}")
#         print(f"pubDate: {pubDate}")
#         print(f"Image: {imgLink}")
#         print("\n")        
        
        
        entry_dict = {
            'title': title,
            'description': description,
            'link': link,
            'pubDate': pubDate,
            'imgLink': imgLink,
            'feedType': feed_type
        }

        
        
        feed_datas.append(entry_dict)
    return feed_datas


        #desc = item.find('description').text.strip()
        #print(desc)


In [50]:
def create_database():
    conn = sqlite3.connect('rss_data.db')
    cursor = conn.cursor()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS toi_recent_popular_news (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            description TEXT,
            link TEXT,
            pubDate TEXT,
            imgLink TEXT,
            feedType TEXT
        )
    ''')
    conn.commit()
    conn.close()

In [64]:
def interact_database_table(entries):
    conn = sqlite3.connect('rss_data.db')
    
    cursor = conn.cursor()
    print(cursor)
    for entry in entries:
        print(entry)
        cursor.execute('''
            INSERT INTO toi_recent_popular_news (title, description, link, pubDate,imgLink, feedType)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            entry.get('title', ''),
            entry.get('description', ''),
            entry.get('link', ''),
            entry.get('pubDate', ''),
            entry.get('imgLink', ''),
            entry.get('feedType', '')
             ))
        
    conn.commit()
    conn.close()
    

In [62]:
def rss_data_in_table(url, rss_type):
    xml_data = fetch_rss_feed(url)
    rss_entries = parse_and_insert(xml_data, rss_type)
    #print(rss_entries)
    interact_database_table(entries=rss_entries)


In [67]:
if __name__ == "__main__":
    #rss_feed_url = "https://timesofindia.indiatimes.com/rssfeedstopstories.cms"
    #rss_feed_url = "https://timesofindia.indiatimes.com/rssfeeds/4719148.cms"
    rss_feed_url = "https://timesofindia.indiatimes.com/rssfeeds/54829575.cms"
    create_database()
    # xml_data = fetch_rss_feed(rss_feed_url)
    # rss_entries = parse_and_insert(xml_data)
    # interact_database_table(rss_entries)
    rss_type = "Sports"
    rss_data_in_table(rss_feed_url, rss_type=rss_type)
        
        
        
    #print(rss_entries)

{'title': 'U19 WC Final Live: Dixon, Weibgen steady after Australia lose Konstas early', 'description': '<img border="0" hspace="10" align="left" style="margin-top:3px;margin-right:5px;" src="https://timesofindia.indiatimes.com/photo/107596568.cms" />', 'link': 'https://timesofindia.indiatimes.com/sports/cricket/india-vs-australia-ind-vs-aus-live-score-under-19-world-cup-final-2024-from-benoni/liveblog/107596568.cms', 'pubDate': '2024-02-11T11:30:16+05:30', 'imgLink': 'https://static.toiimg.com/photo/msid-107596568,imgsize-61186.cms', 'feedType': 'Sports'}
{'title': "'Boomball': Ashwin lauds Vizag 'show-stealer' Bumrah", 'description': '<img border="0" hspace="10" align="left" style="margin-top:3px;margin-right:5px;" src="https://timesofindia.indiatimes.com/photo/107596079.cms" />Despite England taking a 1-0 lead with a victory over India in the first Test at Hyderabad, Bumrah\'s precision yorkers and his ability to apply pressure on batters disrupted England\'s aggressive and result-o